In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import urllib
from collections import deque
tf.compat.v1.disable_eager_execution()

In [14]:
# the number of iterations to train for
numTrainingIters = 10000

# the number of hidden neurons that hold the state of the RNN
hiddenUnits = 1000

# the number of classes that we are learning over
numClasses = 3

# the number of data points in a batch
batchSize = 100

# size of the window
windowSize = 10

# number of filters
numFilters = 8

# learning rate
learning_rate = 0.005

In [3]:
# this function takes a dictionary (called data) which contains
# of (dataPointID, (classNumber, matrix)) entries.  Each matrix
# is a sequence of vectors; each vector has a one-hot-encoding of
# an ascii character, and the sequence of vectors corresponds to
# one line of text.  classNumber indicates which file the line of
# text came from.
#
# The argument maxSeqLen is the maximum length of a line of text
# seen so far.  fileName is the name of a file whose contents
# we want to add to data.  classNum is an indicator of the class
# we are going to associate with text from that file.  linesToUse
# tells us how many lines to sample from the file.
#
# The return val is the new maxSeqLen, as well as the new data
# dictionary with the additional lines of text added
def addToData (maxSeqLen, data, fileName, classNum, linesToUse):
    #
    # open the file and read it in
    response = urllib.request.urlopen(fileName)
    content = response.readlines ()
    #
    # sample linesToUse numbers; these will tell us what lines
    # from the text file we will use
    myInts = np.random.randint (0, len(content), linesToUse)
    #
    # i is the key of the next line of text to add to the dictionary
    i = len(data)
    #
    # loop thru and add the lines of text to the dictionary
    for whichLine in myInts.flat:
        #
        # get the line and ignore it if it has nothing in it
        line = content[whichLine].decode("utf-8")
        if line.isspace () or len(line) == 0:
            continue;
        #
        # take note if this is the longest line we've seen
        if len (line) > maxSeqLen:
            maxSeqLen = len (line)
        #
        # create the matrix that will hold this line
        temp = np.zeros((len(line), 256))
        #
        # j is the character we are on
        j = 0
        #
        # loop thru the characters
        for ch in line:
            #
            # non-ascii? ignore
            if ord(ch) >= 256:
                continue
            #
            # one hot!
            temp[j][ord(ch)] = 1
            #
            # move onto the next character
            j = j + 1
            #
        # remember the line of text
        data[i] = (classNum, temp)
        #
        # move onto the next line
        i = i + 1
    #
    # and return the dictionary with the new data
    return (maxSeqLen, data)

In [4]:
# this function takes as input a data set encoded as a dictionary
# (same encoding as the last function) and pre-pends every line of
# text with empty characters so that each line of text is exactly
# maxSeqLen characters in size
def pad (maxSeqLen, data):
   #
   # loop thru every line of text
   for i in data:
        #
        # access the matrix and the label
        temp = data[i][1]
        label = data[i][0]
        #
        # get the number of chatacters in this line
        len = temp.shape[0]
        #
        # and then pad so the line is the correct length
        padding = np.zeros ((maxSeqLen - len,256))
        data[i] = (label, np.transpose (np.concatenate ((padding, temp), axis = 0)))
   #
   # return the new data set
   return data


In [5]:
# this generates a new batch of training data of size batchSize from the
# list of lines of text data. This version of generateData is useful for
# an RNN because the data set x is a NumPy array with dimensions
# [batchSize, 256, maxSeqLen]; it can be unstacked into a series of
# matrices containing one-hot character encodings for each data point
# using tf.unstack(inputX, axis=2)
def generateDataRNN (maxSeqLen, data, batchSize = batchSize):
    #
    # randomly sample batchSize lines of text
    myInts = np.random.randint (0, len(data), batchSize)
    #
    # stack all of the text into a matrix of one-hot characters
    x = np.stack ([data[i][1] for i in myInts.flat])
    #
    # and stack all of the labels into a vector of labels
    y = np.stack ([np.array((data[i][0])) for i in myInts.flat])
    #
    # return the pair
    return (x, y)

In [6]:
# this also generates a new batch of training data, but it represents
# the data as a NumPy array with dimensions [batchSize, 256 * maxSeqLen]
# where for each data point, all characters have been appended.  Useful
# for feed-forward network training
def generateDataFeedForward (maxSeqLen, data):
    #
    # randomly sample batchSize lines of text
    myInts = np.random.randint (0, len(data), batchSize)
    #
    # stack all of the text into a matrix of one-hot characters
    x = np.stack (data[i][1].flatten () for i in myInts.flat)
    #
    # and stack all of the labels into a vector of labels
    y = np.stack (np.array((data[i][0])) for i in myInts.flat)
    #
    # return the pair
    return (x, y)


In [7]:
# create the data dictionary
maxSeqLen = 0
data = {}

# load up the three data sets
(maxSeqLen, data) = addToData (maxSeqLen, data, "https://s3.amazonaws.com/chrisjermainebucket/text/Holmes.txt", 0, 10000)
(maxSeqLen, data) = addToData (maxSeqLen, data, "https://s3.amazonaws.com/chrisjermainebucket/text/war.txt", 1, 10000)
(maxSeqLen, data) = addToData (maxSeqLen, data, "https://s3.amazonaws.com/chrisjermainebucket/text/william.txt", 2, 10000)

# pad each entry in the dictionary with empty characters as needed so
# that the sequences are all of the same length
data = pad (maxSeqLen, data)



In [8]:
num_test = 3000
print(maxSeqLen, len(data))

79 25046


In [9]:
def split_data(data, num_test):
    """
    Splits the data into training and testing sets based on the specified test ratio.

    Args:
    data (dict): Dictionary containing the data to split.
    test_ratio (float): Proportion of the dataset to include in the test split.

    Returns:
    tuple: Two dictionaries, the first being the training set and the second the test set.
    """
    # Extract all keys (indices) from the data
    keys = list(data.keys())
    np.random.shuffle(keys)  # Randomly shuffle the keys

    # Split the keys into training and testing
    test_keys = keys[:num_test]
    train_keys = keys[num_test:]

    # Create dictionaries for train and test datasets
    train_data = {i: data[key] for i, key in enumerate(train_keys)}
    test_data = {i: data[key] for i, key in enumerate(test_keys)}

    return train_data, test_data


In [10]:
# Splitting padded data into Train and Test Data
train_data, test_data = split_data(data, num_test)  # 25% of data as test set

print("Training Set Size:", len(train_data))
print("Test Set Size:", len(test_data))

Training Set Size: 22046
Test Set Size: 3000


In [11]:
def find_max_seq_len(data):
    """
    Finds the maximum sequence length in a dataset.

    Args:
    data (dict): Dictionary containing the data, where each entry is a tuple (classNumber, matrix).

    Returns:
    int: The maximum sequence length in the dataset.
    """
    max_len = 0
    for key, value in data.items():
        # value[1] is the matrix, and its sum is maxSeqLen as it is one-hot-encoded
        sequence_length = value[1].sum()
        if sequence_length > max_len:
            max_len = sequence_length
    return int(max_len)


In [12]:
maxSeqLen_train = find_max_seq_len(train_data)
maxSeqLen_test = find_max_seq_len(test_data)

print("Maximum length of sequence in Training set:", maxSeqLen_train)
print("Maximum length of sequence in Test set:", maxSeqLen_test)

Maximum length of sequence in Training set: 78
Maximum length of sequence in Test set: 78


In [15]:
# now we build the TensorFlow computation... there are two inputs,
# a batch of text lines and a batch of labels
inputX = tf.placeholder(tf.float32, [batchSize, 256, maxSeqLen])
inputY = tf.placeholder(tf.int32, [batchSize])

# this is the inital state of the RNN, before processing any data
initialState = tf.placeholder(tf.float32, [batchSize, hiddenUnits])

# Creaate filters for the time warp and the current state
# The filters are of size (2560, 8) and (2560, 8) respectively
# The output of the filter is of size (batchSize, 8)
# The filters are initialized with random values
Wfilter1 = tf.Variable(np.random.normal(0, 0.01, (256 * windowSize, numFilters)), dtype=tf.float32)
# Wfilter2 = tf.Variable(np.random.normal(0, 0.01, (256 * windowSize, 8)), dtype=tf.float32)

# the weight matrix that maps the inputs and hidden state to a set of values
Wfir = tf.Variable(np.random.normal(0, 0.01, (hiddenUnits + hiddenUnits + numFilters, hiddenUnits)), dtype=tf.float32)
Wsec = tf.Variable(np.random.normal(0, 0.01, (hiddenUnits, hiddenUnits)), dtype=tf.float32)

# weights and bias for the final classification
W2 = tf.Variable(np.random.normal (0, 0.05, (hiddenUnits, numClasses)),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,numClasses)), dtype=tf.float32)

# unpack the input sequences so that we have a series of matrices,
# each of which has a one-hot encoding of the current character from
# every input sequence
sequenceOfLetters = tf.unstack(inputX, axis=2)

def getWindows(x, maxSeqLen, windowSize):
    windows = []
    for i in range(maxSeqLen - windowSize + 1):
        window = x[:, :, i:i+windowSize]
        # merge innermost elements
        window = tf.reshape(window, (window.shape[0], -1))
        windows.append(window)
    return windows

sequenceOfWindows = getWindows(inputX, maxSeqLen, windowSize)

# now we implement the forward pass
currentState = initialState
window = deque()
window.extend([initialState]*10)
for idx, timeTick in enumerate(sequenceOfWindows):

    convTimeTick = tf.matmul(timeTick, Wfilter1)

    timeWarpState = window.popleft()

    inputPlusState = tf.concat([convTimeTick, currentState, timeWarpState], 1)

    next_state = tf.tanh(tf.matmul(inputPlusState, Wfir))

    last_state = tf.tanh(tf.matmul(next_state, Wsec))


    window.append(next_state)

    currentState = last_state

# compute the set of outputs
outputs = tf.matmul(currentState, W2) + b2

predictions = tf.nn.softmax(outputs)

# compute the loss
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=outputs, labels=inputY)
totalLoss = tf.reduce_mean(losses)

# use gradient descent to train
trainingAlg = tf.compat.v1.train.AdagradOptimizer(learning_rate).minimize(totalLoss)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
# and train!!
with tf.Session() as sess:
    #
    # initialize everything
    sess.run(tf.compat.v1.global_variables_initializer())
    #
    # and run the training iters
    for epoch in range(numTrainingIters):
        #
        # get some data
        x, y = generateDataRNN (maxSeqLen, train_data,  batchSize=100)
        #
        # do the training epoch
        _currentState = np.zeros((batchSize, hiddenUnits))
        _totalLoss, _trainingAlg, _currentState, _predictions, _outputs = sess.run(
                [totalLoss, trainingAlg, currentState, predictions, outputs],
                feed_dict={
                    inputX:x,
                    inputY:y,
                    initialState:_currentState
                })
        #
        # just FYI, compute the number of correct predictions
        numCorrect = 0
        for i in range (len(y)):
           maxPos = -1
           maxVal = 0.0
           for j in range (numClasses):
               if maxVal < _predictions[i][j]:
                   maxVal = _predictions[i][j]
                   maxPos = j
           if maxPos == y[i]:
               numCorrect = numCorrect + 1
        #
        # print out to the screen
        print("Step", epoch, "Loss", _totalLoss, "Correct", numCorrect, "out of", batchSize)

    Numtestiter = num_test // batchSize
    test_correct = 0
    totalTestLoss = 0
    xtest, ytest = generateDataRNN(maxSeqLen, test_data, batchSize=num_test)
    for idx in range(Numtestiter):
        _currentState = np.zeros((batchSize, hiddenUnits))
        _totalLoss, _predictions = sess.run(
                [totalLoss, predictions],
                feed_dict={
                    inputX:xtest[(idx*batchSize):((idx+1)*batchSize)],
                    inputY:ytest[(idx*batchSize):((idx+1)*batchSize)],
                    initialState:_currentState
                })

        numCorrect = 0
        for i in range (batchSize):
            maxPos = -1
            maxVal = 0.0
            for j in range (numClasses):
                if maxVal < _predictions[i][j]:
                    maxVal = _predictions[i][j]
                    maxPos = j
            if maxPos == ytest[i]:
                numCorrect = numCorrect + 1

            test_correct = test_correct + numCorrect
        totalTestLoss += _totalLoss * batchSize

    print(f"Loss for {num_test} test samples is {totalTestLoss / num_test} and correct is {test_correct} out of {num_test}")


Streaming output truncated to the last 5000 lines.
Step 5001 Loss 0.2953292 Correct 88 out of 100
Step 5002 Loss 0.39324322 Correct 81 out of 100
Step 5003 Loss 0.4697617 Correct 75 out of 100
Step 5004 Loss 0.45064843 Correct 77 out of 100
Step 5005 Loss 0.38710967 Correct 78 out of 100
Step 5006 Loss 0.43724725 Correct 77 out of 100
Step 5007 Loss 0.33426574 Correct 85 out of 100
Step 5008 Loss 0.40452155 Correct 85 out of 100
Step 5009 Loss 0.4849668 Correct 80 out of 100
Step 5010 Loss 0.4027681 Correct 83 out of 100
Step 5011 Loss 0.42728126 Correct 83 out of 100
Step 5012 Loss 0.30882493 Correct 85 out of 100
Step 5013 Loss 0.4332822 Correct 78 out of 100
Step 5014 Loss 0.38610762 Correct 79 out of 100
Step 5015 Loss 0.35547054 Correct 86 out of 100
Step 5016 Loss 0.312221 Correct 82 out of 100
Step 5017 Loss 0.33156246 Correct 84 out of 100
Step 5018 Loss 0.4545347 Correct 83 out of 100
Step 5019 Loss 0.48133743 Correct 71 out of 100
Step 5020 Loss 0.5285841 Correct 75 out of 10